In [2]:
import os
import zipfile
import pandas as pd

In [6]:
# Read norman and sirius identification files produced. Also read feature finder correlation file
corr_cutoff = 0.75
input_sirius_file = "analysis_results/cov0.9_corr"+str(corr_cutoff)+".sirius"
norman_identification_file = "analysis_results/coverage0.9_correlation"+str(corr_cutoff)+"_identification.tsv"
found_feature_file = "analysis_results/coverage0.9_correlation"+str(corr_cutoff)+".csv"

In [7]:
# Extract a given sirius file to a folder "temp"
# Extract at the location of the sirius file
# Used along with compare_fp_from_sirius function
def extract_sirius_file(sirius_file,folder_name="temp"):
    # Extract and search, use for small files. Pretty memory and I/O intensive
    path = os.path.dirname(os.path.abspath(sirius_file)) + "/"
    zf = zipfile.ZipFile(sirius_file)
    try:
        os.mkdir(path + folder_name)
    except:
        pass
    zf.extractall(path + folder_name)
    zf.close()

def get_sirius_results(sirius_file):
    extract_sirius_file(sirius_file)
    true_hits = []
    path = os.path.dirname(os.path.abspath(sirius_file)) + "/"
    all_structure_file = path + 'temp/compound_identifications.tsv'
    output_df = pd.read_csv(all_structure_file,sep="\t")
    os.system("rm -r "+ path + "temp")
    return  output_df

In [8]:
# Read the files as a dataframe
sirius_df = get_sirius_results(input_sirius_file)
norman_id_df = pd.read_csv(norman_identification_file,sep="\t") 
ff_df = pd.read_csv(found_feature_file) 

In [9]:
# Left join the files to produce all the results in once single file
ff_norman_df = ff_df.merge(norman_id_df.rename({'met_id': 'Met_ID'}, axis=1),
               left_on='Met_ID', right_on='Met_ID', how='left')

sirius_ff_normal_df = ff_norman_df.merge(sirius_df.rename({'ionMass': 'mz_cf_x'}, axis=1),
               left_on='mz_cf_x', right_on='mz_cf_x', how='left')

consolidated_results = sirius_ff_normal_df[['Unnamed: 0_x', 'Met_ID', 'rt_cf_x', 'mz_cf_x', 'intensity_cf_x',
       'label_x', 'real_name_x', 'log_k', 'pred_log_k_value','names',
       'adduct', 'name', 'smiles', 'links', 'id']]
consolidated_results.rename({'Unnamed: 0_x': 'Unnamed',
                             'rt_cf_x': 'rt_cf_ff',
                             'mz_cf_x': 'mz_cf_ff',
                             'intensity_cf_x': 'intensity_cf_ff',
                            'label_x': 'label_clustering',
                            'real_name_x': 'real_name_inhouse',
                            'log_k': 'log_k_norman',
                            'adduct': 'adduct_sirius',
                            'name': 'name_sirius',
                            'smiles': 'smiles_sirius',
                            'links': 'links_sirius',
                            'id': 'id_sirius',
                            'Met_ID':'Met_ID_ff',
                            'pred_log_k_value':'pred_log_k_value_lm',
                            'names':'names_norman'}, axis=1,inplace=True)

/tmp/ipykernel_154051/2311950403.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consolidated_results.rename({'Unnamed: 0_x': 'Unnamed',


In [11]:
# Save the results in a CSV
consolidated_results.to_csv("analysis_results/cov0.9_corr" + str(corr_cutoff) + "_consolidated.csv")